## Introduction:

The IceCube Neutrino Observatory is composed of many light detectors, called "DOMs" (digital optical modules), buried two kilometers deep below the surface of the South Pole. These light detectors are arranged in a roughly three dimensional grid: there are many "strings" hanging vertically downwards in the ice, and on each string there are many DOMs: 

![](../resources/images/I3det_v2_edited.jpeg)

When a high energy charged particle passes through the ice, it produces what is called Cherenkov light, which can then be detected by the many DOMs. The **locations** of the DOMs that see light, the **times** at which they saw light, and the **amount** of light they see all communicate information about this charged particle.

As experimentalists, what we want to do is work backwards: if what we have is this information about the light seen by the detector, what can we figure out about the original particle? 

## 1: Visualizing events:

For this activity, we've prepared a simulation of many events within the IceCube detector. Execute the cell below by clicking the triangle or hitting `shift return` to download the simulation from google drive. Google CoLab will ask you if you trust the notebook: hit `Run Anyway`. 

In [ ]:
# needed for figures to appear in colab 
from google.colab import output
output.enable_custom_widget_manager()
# from IPython.display import display, clear_output

!rm -rf sample_data
!mkdir sim_moonshadow

# download tracks.parquet
!gdown 1nqffw6xHLdX2oO8d-5xjExYZryo7rp3x
!mv tracks.parquet sim_moonshadow/tracks.parquet

# # download cascades.parquet
# !gdown 1yo3jD0a9xB2FfIXJJuMRc71IMe6nztvq
# !mv cascades.parquet sim_moonshadow/cascades.parquet

# download pre-prepared analysis code
!rm -rf IceCube_MasterClass_at_Harvard2024
!git clone "https://github.com/kcarloni/IceCube_MasterClass_at_Harvard2024";

import sys
sys.path.insert(0, "./IceCube_MasterClass_at_Harvard2024/")

In [2]:
from src.event_reader import load_sim_events

tracks = load_sim_events("sim_moonshadow/tracks.parquet")
# tracks = load_sim_events("../../sim_prometheus/reduced_muons.parquet")
# tracks = EventSelection("tracks.parquet")

Let's look at an 'event' in the detector. An event is just a bunch of 'hits', ie. instances in which light hit different sensors, which are grouped together because they all occured within the same time window.

In [3]:
evt_num = 24
evt = tracks[evt_num]
evt

We can visualize an event within the detector by plotting it.  

In [4]:
import plotly.graph_objs as go
from src.plot_event import *

layout = get_3d_layout()
plot_det = plot_I3det("../../")

fig = go.FigureWidget(data=plot_det, layout=layout)

plot_evt = plot_first_hits(evt)
fig.add_trace(plot_evt)

fig.show()

## 2: From what direction did the neutrino come from? 

In the event displays we've just looked at, redder hits occur earlier and greener hits occur after. Given this information, can you figure out in what direction the neutrino was going?

We've written a game below to see how well you can do. Execute the cell below (`shift return`) to start the game. 

You should see a similar event display, along with a big arrow, which represents your guess for the direction of the neutrino. Adjust this arrow using the zenith and azimuth angle sliders provided. Once you are happy with your guess, hit the submit button below the event display. The event display will be refreshed to show the actual true path of the neutrino, and how many degrees you were off the actual path will be printed below. Try to be as close as possible to the actual path, and minimize the degrees you were off by!

After submitting a guess, you can return to the game by clicking the Return button. You can then try to guess again on the same event, or try your hand at a different one by opening the event_id dropdown menu and selecting a different one.

WIP: Felix's directional reconstruction game. 

In [ ]:
# !pip install "analysis @ git+https://github.com/jlazar17/IceCube_Masterclass_MoonShadow";

# from analysis import DataReader
# from analysis import reco_game

# tracks = DataReader("tracks.parquet")

In [ ]:
from src.reco_game import reco_game
reco_game( tracks )

As we discussed earlier today, IceCube can see two broad kinds of events, *tracks*, which look like long lines and are produced by the light from *muons*, and *cascades*, which look like spherical blobs and are produced by the light from *electrons* (and some other particles). 

We just tried to reconstruct the direction of some track events. Do you think it would be easier or harder to reconstruct the direction of some cascade events? 

Give it a try!

In [ ]:
cascades = DataReader("cascades.parquet")
reco_game(cascades)

## 3: Can we use computers to do a better job at figuring out the direction?

When we try to figure out the direction in the game below, what we're actually doing is trying to get the direction arrow to go as close to through the middle of all the hits as possible.

We can quantify this sense of "how good" with some math! Imagine drawing straight lines from each hit to the arrow, like in the image below:

the more of these lines are shorter, the better our guess of the direction. We can calculate the length of each line using some *vector* math:

We can also write a *function* in python to do this calculation for us! Just like in math, this *function* takes in some inputs (the direction vector and the hit) and produces one output (the perpendicular distance). 

In [5]:
# def get_direction_vector_from_angles( azi, zen ):
#     return np.array([
#         np.cos(azi) * np.sin(zen),
#         np.sin(azi) * np.sin(zen),
#         np.cos(zen)
#     ])

# def get_direction_angles_from_vector( dir ):
#     azi = np.arctan2( dir[1], dir[0] )
#     zen = np.arccos( dir[2] )
#     return np.array([azi, zen])

from src.direction_utils import *

In [6]:
"""
"""
def calc_perpendicular_distance( hit_pt, dir_vec, pivot_pt ):
    dist_vec = hit_pt - pivot_pt
    return np.sqrt( np.dot( dist_vec, dist_vec ) -  np.dot( dist_vec, dir_vec )**2 )

"""
"""
def calc_mean_perpendicular_distance( dir_angles, hit_pts, pivot_pt ):
    
    dir_vec = get_direction_vector_from_angles( dir_angles[0], dir_angles[1] )
    return np.mean([ calc_perpendicular_distance(hit, dir_vec, pivot_pt) for hit in hit_pts ])

In [10]:
def calc_center_of_gravity( hits ):
    return hits.mean(axis=0)

    # hits = evt.hits[["t", "sensor_pos_x", "sensor_pos_y", "sensor_pos_z"]].to_numpy()
    # return hits.mean(axis=0)[1:4]

Let's check that bad guesses really result in a larger *mean*, or average, perpendicular distance. Adjust the values of the zenith `zen` and azimuth `azi` angles below and see how the mean distance changes. We'll also plot the direction vector again for a visual check on the 'goodness' of the guess.

Does the mean distance calculation help you refine your guess more easily? 

In [11]:
# you can change which event you want to look at here: 
evt_num = 5
evt = tracks[evt_num]

In [13]:
# adjust these!
azi = np.deg2rad( 100 )
zen = np.deg2rad( -10 )

# we compute the direction and the pivot point:
pivot_pt = calc_center_of_gravity( evt.hits_xyz )

# and we can then compute the perpendicular distance between the line and each hit, and take the mean. 
mean_dist = calc_mean_perpendicular_distance( 
    np.array( [azi, zen] ),
    evt.hits_xyz, 
    pivot_pt
)

# let's print it!
print( f"the mean distance is {mean_dist:.2f} meters \n" )

# and make a plot:
dir_vec = get_direction_vector_from_angles( azi, zen )

fig = go.FigureWidget( data=plot_det, layout=layout )
plot_evt = plot_first_hits( evt )
fig.add_trace( plot_evt )
fig.add_traces( plot_direction( dir_vec, pivot_pt, color="dodgerblue" ) )
fig.show()

the mean distance is 88.24 meters 



Just like you *minimized* the mean perpendicular distance by adjusting the zenith and azimuth values, a computer can minimize it using a minimization algorithm. Such algorithms define a procedure by which the computer repeatedly guesses some parameter (zenith and azimuth) values, evaluates the goodness of the guess (computes the mean distance), and then makes a new guess, over and over until some stopping condition (a condition that indicates that we think we've found the minimum).

The python `scipy` library, for scientific programming, offers many different minimization algorithms in its `scipy.optimize` module. Let's try using one! 

In [ ]:
from scipy.optimize import minimize


The `minimize` function takes in two inputs: 
- the first input is a function f whose input is a list of the parameters you want to minimize
- the second input is your initial guess for those parameters.

In [1]:
def function_to_minimize( dir_angles ):
    return calc_mean_perpendicular_distance( dir_angles, evt.hits_xyz, pivot_pt )


def make_initial_guess_for_angles( evt ):

    # initial_guess_azi = np.deg2rad( -10 )
    # initial_guess_zen = np.deg2rad( 180 )
    # return np.array( [initial_guess_azi, initial_guess_zen] )
    
    j0, j1 = np.argmin( evt.hits_t ), np.argmax( evt.hits_t )
    initial_guess = get_direction_angles_from_vector(
        normalize( evt.hits_xyz[j1, :] - evt.hits_xyz[j0, :] )
    )
    return initial_guess

In [ ]:
# let's run the minimization!
minimization_output = minimize(
    function_to_minimize,
    make_initial_guess_for_angles( evt ),
)

best_guess_azi, best_guess_zen = minimization_output.x
smallest_distance = minimization_output.fun

# let's print the output: 
print( "The angles with the smallest perpendicular distance are:" )
print( f"azi = {np.rad2deg(best_guess_azi):.2f} degrees")
print( f"zen = {np.rad2deg(best_guess_zen):.2f} degrees")
print( "For these angles, the mean perpendicular distance is: \n")
print( f"\t {smallest_distance:.2f} meters")

# and let's plot the direction angle! 
dir_vec = get_direction_vector_from_angles( best_guess_azi, best_guess_zen )

fig = go.FigureWidget(data=plot_det, layout=layout)
plot_evt = plot_first_hits( evt )
fig.add_trace( plot_evt )
fig.add_traces( plot_direction( dir_vec, pivot_pt, color="hotpink" ) )
fig.show()

<!-- Take a look at the plotted arrow. Is it going in the right direction?  -->

Let's also check the performance of our computer against the true answer.

(We can do this for these events because they are all simulated, so we know their true properties. If we were doing this with a real event from data collected by IceCube, we would not have any way of knowing this real right answer. This is why simulation is so important -- it gives us a way of checking how good we are at guessing the true quantities. )

In [ ]:
true_dir_vec = get_direction_vector_from_angles( 
    evt.true_muon_azimuth, 
    evt.true_muon_zenith )

fig.add_traces( plot_direction( true_dir_vec, pivot_pt, color="black" ) )
fig.show()

## 4. How can we quantify the quality of our "reconstructed" directions?

Let's say we have more than one method to determine the direction of an event. How can we decide which one is better? 

If you haven't yet, go back to step 3. and try changing which event you figure out the direction for. 
- Are there any kinds of events which are easier / harder for you to guess? 
- Are there any kinds of events which are easier / harder for the computer algorithm to reconstruct? 

As you may see, how well we do at figuring out the direction varies a lot between different events. If we want to pick the best method, we need to look at how well it does for many different events.

Let's start by using the computer algorithm to reconstruct the directions of many events. To do this fast, we'll use the code from above to write a function whose input is an event and whose output is the best guess direction. 

In [ ]:
def find_best_dir( evt ):

    pivot_pt = calc_center_of_gravity( evt )
    
    # you can define functions inside of other functions!
    def function_to_minimize( dir_angles ):
        return calc_mean_perpendicular_distance( dir_angles, evt.hits_xyz, pivot_pt )

    initial_guess = make_initial_guess_for_angles( evt )

    out = minimize( 
        function_to_minimize,
        initial_guess, 
        method='Nelder-Mead'
    )

    best_azi, best_zen = out.x

    # # these functions just make sure that the 
    # best_azi = bound_azi( best_azi )
    # best_zen = bound_zen( best_zen )

    return np.array([best_azi, best_zen])


Let's use this function to reconstruct the directions of a bunch of events. 

In [ ]:
# change this number to adjust how many events you want to use! 
N = 100

# we create empty arrays to hold the reconstructed angles.
reco_azi = np.empty( N )
reco_zen = np.empty( N )

# let's also save the true angles. 
true_azi = np.empty( N )
true_zen = np.empty( N )

# now we iterate over all the events...
for evt_num in range(N):
    
    evt = tracks[evt_num]
    reco_azi[evt_num], reco_zen[evt_num] = find_best_dir( evt )

    true_azi[evt_num] = evt.true_muon_azimuth
    true_zen[evt_num] = evt.true_muon_zenith


The first thing we can do is to check the average differences. First we'll write some functions to calculate the difference between two azimuths or two zeniths. 

In [ ]:
def calc_zenith_diff( zen_1, zen_2 ):
  return np.abs( zen_1 - zen_2 )

def calc_azimuth_diff( azi_1, azi_2 ):

  # this is a little trickier, because 0° and 360° (or, in radians, 0 and 2pi) mean the same thing!
  # we need to remember that the biggest possible difference between two angles is 180° (pi radians).
  
  abs_diff =  np.abs( azi_1 - azi_2 )
  return np.minimum( abs_diff, 2*np.pi - abs_diff )

In [ ]:
# now we can iterate over all our events and compute the differences!
# diff_azi = np.empty( N )
# diff_zen = np.empty( N )

# for evt_num in range(N):
#     diff_azi[evt_num] = calc_azimuth_diff( true_azi[evt_num], reco_azi[evt_num] )
#     diff_zen[evt_num] = calc_zenith_diff( true_zen[evt_num], reco_zen[evt_num] )


diff_azi = calc_azimuth_diff( true_azi, reco_azi )
diff_zen = calc_azimuth_diff( true_zen, reco_zen )

print("the average difference between \n")
print(f"\t the true and reconstructed azimuth angles is {np.rad2deg(diff_azi).mean():.2f}°" )
print(f"\t the true and reconstructed zenith angles is {np.rad2deg(diff_zen).mean():.2f}°" )

Are these what you expected?

We can investigate more closely by making what is called a histogram. A histogram essentially counts how many items in a collection have values that fall in different bins. 

For example, a histogram of the *differences between the true and reco zenith angles* for many events might say that there were 10 events for which the difference was less than 10°, 30 for which it was between 10° and 20°, 5 for which it was between 20° and 30°, ...